In [148]:
import pandas as pd
import numpy as np
import csv

# 导入tsv数据

In [2]:
def read_from_tsv(file_path: str, column_names: list) -> list:
    csv.register_dialect('tsv_dialect', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(file_path, "r",encoding='utf-8') as wf:
        reader = csv.DictReader(wf, fieldnames=column_names, dialect='tsv_dialect')
        datas = []
        for row in reader:
            data = dict(row)
            datas.append(data)
    csv.unregister_dialect('tsv_dialect')
    return datas

In [3]:
train_data = read_from_tsv('train.tsv',['Uid','Date', 'History', 'Impression'])[1:]
train_news_data = read_from_tsv('train_news.tsv',['Nid','Category', 'SubCategory', 'Title', 'Abstract'])[1:]
test_data = read_from_tsv('test.tsv',['Uid','Date', 'History', 'Impression'])[1:]
test_news_data = read_from_tsv('test_news.tsv',['Nid','Category', 'SubCategory', 'Title', 'Abstract'])[1:]

In [4]:
data = train_data + test_data
news_data = train_news_data + test_news_data

In [5]:
train_data[:2]

[{'Uid': 'U48196',
  'Date': '11/11/2019 7:56:24 PM',
  'History': 'N51591 N23288 N33604 N3908 N16730 N23571 N16730 N44018 N5844 N23979 N48369 N37706 N15745 N28487 N43353 N54529 N23488 N3682 N26318 N18896 N6004 N11701 N10503 N29088 N719 N37264 N54069 N30026 N871 N27167 N37378 N6233 N51661 N33115 N29101 N26978 N24920 N59754 N31322 N20039 N13603 N33200 N62705 N4394 N3680 N50513 N11175 N1088 N47525 N35820 N41351 N1603 N34198 N49229 N31931 N59763 N37127 N22081 N63376 N56693 N49458 N54827 N274 N9803 N25905 N26272 N3613 N59359 N48259 N21178 N51541 N24492 N26503 N29742 N55743 N2086 N28679 N38299 N10848 N3130 N41997 N23045 N7851 N19280 N57760 N17542 N60360 N64609 N5351 N34000 N43142 N58391 N33742 N7853 N14385 N39161 N47482 N26122 N48356 N11231 N53422 N7019 N26826 N17050 N14761 N25258 N24409 N28856 N61355 N39414 N16922 N15663 N10814 N6785 N55001 N32004 N62211 N25912 N13753 N54072 N7646 N55055 N6767 N31281 N22012 N22816 N15676 N41881 N34520 N1294',
  'Impression': 'N58410-0 N53696-0 N51797-0 N64

In [6]:
train_news_data[:2]

[{'Nid': 'N55528',
  'Category': 'lifestyle',
  'SubCategory': 'lifestyleroyals',
  'Title': 'The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By',
  'Abstract': "Shop the notebooks, jackets, and more that the royals can't live without."},
 {'Nid': 'N19639',
  'Category': 'health',
  'SubCategory': 'weightloss',
  'Title': '50 Worst Habits For Belly Fat',
  'Abstract': 'These seemingly harmless habits are holding you back and keeping you from shedding that unwanted belly fat for good.'}]

In [7]:
test_data[:2]

[{'Uid': 'U21693',
  'Date': '11/15/2019 6:52:40 AM',
  'History': 'N770 N50047 N4020 N306 N64467 N4607 N41449',
  'Impression': 'N11930 N7419 N60747 N41946 N7342 N23513 N31958 N62318 N5940 N36940 N36779 N43646 N51793 N6916 N53242 N30290 N23767 N42844 N42233 N35216 N48487 N13408 N10423 N46976 N42767 N13865 N46162 N32536 N20187 N36786 N27738 N27057 N24109 N6638 N40656 N20036 N13556 N42670'},
 {'Uid': 'U25377',
  'Date': '11/15/2019 5:44:39 AM',
  'History': 'N24302 N6385 N49997 N33371 N47121 N11917 N52096 N10646 N60595 N10059 N54932 N37669 N35637 N28467 N4830 N11701 N11701 N4830 N35637 N59231 N17176 N14340 N54099 N21260 N53520 N13669 N2939 N54959 N58715 N29249 N58091 N21851 N19638 N48492 N13605 N47525 N55310 N61681 N39235 N59496 N63239 N49159 N61681 N64049 N44598 N61018 N4060 N63633 N38865 N36691 N7158 N43132 N27424 N40704 N43132 N26045 N8031 N28144 N49728 N43903 N61681 N23157 N3057 N17165 N23157 N28550 N61388 N27830 N49325 N59359 N40692 N33073 N32836 N59691 N16480 N26758 N5344 N43380 N

In [8]:
test_news_data[:2]

[{'Nid': 'N55528',
  'Category': 'lifestyle',
  'SubCategory': 'lifestyleroyals',
  'Title': 'The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By',
  'Abstract': "Shop the notebooks, jackets, and more that the royals can't live without."},
 {'Nid': 'N18955',
  'Category': 'health',
  'SubCategory': 'medical',
  'Title': "Dispose of unwanted prescription drugs during the DEA's Take Back Day",
  'Abstract': ''}]

# 基于内容的DNN模型

## 生成数据

In [9]:
# feature
# 每一个feature实际上是其各种浏览内容的几个部分组成：
#1.最关注的3个Category 
#2.Abstract与Title进行词过滤后最热的n个词汇，并进行word emdbedding

# 生成类别集合，方便下面数据转换
news_Category_set = []
for i in train_news_data:
    news_Category_set.append(i['Category'])
for i in test_news_data:
    news_Category_set.append(i['Category'])
news_Category_set = list(set(news_Category_set))
news_Category_set 

['kids',
 'northamerica',
 'lifestyle',
 'sports',
 'news',
 'video',
 'travel',
 'music',
 'games',
 'weather',
 'entertainment',
 'finance',
 'autos',
 'health',
 'movies',
 'tv',
 'foodanddrink',
 'middleeast']

In [10]:
#生成feature1，Category
feature_c_m = []
for i in train_data[:200]:
    m = []#中介容器,每一个数据清空一次
    for j in i['History'].split(' '):
        for h in range(len(news_data)):
            if j == news_data[h]['Nid'].strip(' '):
                j = news_data[h]['Category']
            else:
                pass
        m.append(news_Category_set.index(j))
    feature_c_m.append(m)

In [11]:
# 筛选出出现最高的三个类型
def select_3(arr):
    result = {}
    for i in set(arr):
        result[i] = arr.count(i)
    d_order=sorted(result.items(),key=lambda x:x[1],reverse=False)
    if len(d_order)>=3:
        a =  [d_order[-3][0],d_order[-2][0],d_order[-1][0]]
    else:
        a = [100,100,100]
    return a

In [12]:
feature_c = []
for i in feature_c_m:
    feature_c.append(select_3(i))

In [13]:
#生成feature2，词嵌入
feature_w_m = []
for i in data[:200]:
    m = ''#中介容器,每一个数据清空一次
    for j in i['History'].split(' '):
        for h in range(len(news_data)):
            if j == news_data[h]['Nid'].strip(' '):
                a = news_data[h]['Abstract']
                b = news_data[h]['Title']
            else:
                pass
        m = a+b
    feature_w_m.append(m)

In [14]:
from gensim.models import word2vec
import gensim
import jieba
import jieba.analyse
words_ls = []
for i in feature_w_m:
    words = jieba.analyse.extract_tags(i, topK=5)
    words_ls.append(words)
#model = word2vec.Word2Vec(words_ls,size=12,window=2,min_count=1)
#model.save('moxing.model')
model = gensim.models.Word2Vec.load('moxing.model')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/qn/mjlm3mc55w13qcd782l_k2r80000gn/T/jieba.cache
Loading model cost 0.585 seconds.
Prefix dict has been built successfully.


In [15]:
len(feature_c)

200

In [16]:
feature_w = []

for m in words_ls[0:len(train_data[:200])]:
    mid3 = []
    mid3.append(model.wv[m[0]].tolist())
    mid3.append(model.wv[m[1]].tolist())
    mid3.append(model.wv[m[2]].tolist())
    mid3.append(model.wv[m[3]].tolist())
    mid3.append(model.wv[m[4]].tolist())
    feature_w.append(np.array(mid3).reshape(60).tolist())

In [17]:
# 生成label，并产生与之对应的feature
feature_m = []
label_m = []
for i in train_data[:200]:
    #print(train_data.index(i))
    feature_mm = feature_c[train_data.index(i)] + list(feature_w[train_data.index(i)])
    for j in i['Impression'].split(' '):
        feature_m.append(np.array(feature_mm))
        label_m.append(int(j[-1]))

In [18]:
len(feature_m)

7576

In [19]:
n = 0
m = 0
feature = []
label = []
for i in range(len(feature_m)):
    if (label_m[i] == 0) and (n>=30) and m<950:
        feature.append(feature_m[i])
        label.append(label_m[i])
        m+=1
    elif label_m[i] == 1 and n<30:
        feature.append(feature_m[i])
        label.append(label_m[i])
        n+=1
    else:
        pass

In [20]:
len(feature)

980

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, Sequential, metrics,models,Input

def get_tensor(feature_list, label_list):
    x = tf.convert_to_tensor(feature_list)
    y = tf.convert_to_tensor(label_list)
    return x,y

def preprocess(x,y):
    #归一化
    x = tf.cast(x,dtype=tf.float32)
    y = tf.cast(y, dtype=tf.float32)
    return x,y

x_train, y_train = get_tensor(feature, label)
#载入训练数据集
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# # shuffle:打乱数据,map:数据预处理，batch:一次取喂入10样本训练
db_train = db_train.shuffle(500).map(preprocess).batch(10)

## 搭建模型

In [22]:
network = models.Sequential()
network.add(layers.Dense(16, activation='relu'))
network.add(layers.Dense(8))
network.add(layers.Dense(1))
network.build(input_shape=(None,63))
optimizer = tf.keras.optimizers.RMSprop(0.01)
network.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])  # 指定评价指标为准备率
# 模型训练
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                1024      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 模型装配
network.fit(db_train,epochs=10)
network.save('my_model.h5')

Epoch 1/10
98/98 [==============================] - 0s 685us/step - loss: 1.3960 - mae: 0.4470 - mse: 1.3960
Epoch 2/10
98/98 [==============================] - 0s 695us/step - loss: 0.0882 - mae: 0.1041 - mse: 0.0882
Epoch 3/10
98/98 [==============================] - 0s 732us/step - loss: 0.0365 - mae: 0.0720 - mse: 0.0365
Epoch 4/10
98/98 [==============================] - 0s 698us/step - loss: 0.0305 - mae: 0.0615 - mse: 0.0305
Epoch 5/10
98/98 [==============================] - 0s 618us/step - loss: 0.0311 - mae: 0.0634 - mse: 0.0311 
Epoch 6/10
98/98 [==============================] - 0s 709us/step - loss: 0.0306 - mae: 0.0606 - mse: 0.0306
Epoch 7/10
98/98 [==============================] - 0s 627us/step - loss: 0.0300 - mae: 0.0597 - mse: 0.0300 
Epoch 8/10
98/98 [==============================] - 0s 719us/step - loss: 0.0305 - mae: 0.0614 - mse: 0.0305 
Epoch 9/10
98/98 [==============================] - 0s 689us/step - loss: 0.0299 - mae: 0.0586 - mse: 0.0299     
Epoch 10/10

## 模型应用

In [24]:
len(test_data)

2000

In [25]:
len(data)

12000

n = 2000
count=0
ls = []
for i in range(n):
    count+=len(test_data[i]['Impression'].split(' '))
    ls.append(len(test_data[i]['Impression'].split(' ')))

In [26]:
#count

In [27]:
ls

Assignment-2.pdf           test_news.tsv
CODE.ipynb                 train/
moxing.model               train.tsv
my_model.h5                train_news.tsv
test/                      第二次作业说明.pdf
test.tsv


In [28]:
#生成feature1，Category
feature_c_m = []
for i in test_data[:len(data)]:
    m = []#中介容器,每一个数据清空一次
    for j in i['History'].split(' '):
        for h in range(len(news_data)):
            if j == news_data[h]['Nid'].strip(' '):
                j = news_data[h]['Category']
            else:
                pass
        m.append(news_Category_set.index(j))
    feature_c_m.append(m)

feature_c = []
for i in feature_c_m:
    feature_c.append(select_3(i))

KeyboardInterrupt: 

In [ ]:
#生成feature2，词嵌入
feature_w_m = []
for i in test_data[:len(data)]:
    m = ''#中介容器,每一个数据清空一次
    for j in i['History'].split(' '):
        for h in range(len(news_data)):
            if j == news_data[h]['Nid'].strip(' '):
                a = news_data[h]['Abstract']
                b = news_data[h]['Title']
            else:
                pass
        m = a+b
    feature_w_m.append(m)
    
for i in feature_w_m:
    words = jieba.analyse.extract_tags(i, topK=5)
    words_ls.append(words)
feature_w = []

In [ ]:
for m in words_ls[0:len(train_data[:count])]:
    mid3 = []
    mid3.append(model.wv[m[0]].tolist())
    mid3.append(model.wv[m[1]].tolist())
    mid3.append(model.wv[m[2]].tolist())
    mid3.append(model.wv[m[3]].tolist())
    mid3.append(model.wv[m[4]].tolist())
    feature_w.append(np.array(mid3).reshape(60).tolist())

In [ ]:
# 生成label，并产生与之对应的feature
feature_m = []
label_m = []
for i in train_data[:count]:
    #print(train_data.index(i))
    feature_mm = feature_c[train_data.index(i)] + list(feature_w[train_data.index(i)])
    for j in i['Impression'].split(' '):
        feature_m.append(np.array(feature_mm))
        label_m.append(int(j[-1]))

x_test, y_test = get_tensor(feature_m, label_m)

In [ ]:
def p(x):
    y = tf.convert_to_tensor(x)
    z = tf.cast(y,dtype=tf.float32)
    return z

def panduan(x):
    if x>0.5:
        x=1
    else:
        x=0
    return x

result = []
for i in range(142):
    result.append(panduan(network.predict(p([x_test[i]]))))

In [ ]:
result

In [ ]:
n = 0
appends = []
for i in ls:
    appends.append(result[n:n+i])
    n = n + i
appends

In [ ]:
def ls2str(ls):
    m = ''
    for i in ls:
        m+=i
    return m
new_data = []
columns = []
for i in range(len(appends)):
    a = test_data[i]
    a['test'] = ls2str(appends[i])
    new_data[].append(a) 
    columns.append(i)

In [ ]:
def write_to_tsv(output_path: str, file_columns: list, data: list):
    csv.register_dialect('tsv_dialect', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(output_path, "w", newline="") as wf:
        writer = csv.DictWriter(wf, fieldnames=file_columns, dialect='tsv_dialect')
        writer.writerows(data)
    csv.unregister_dialect('tsv_dialect')

In [ ]:
write_to_tsv('result.tsv',columns,new_data)

# 打印查看结果

In [179]:
p = pd.read_csv('test2.csv', sep='\t', header=0, usecols=[4])
for j in range(0,2000):
    pre_list = p.loc[j]
    pre_list[0] = pre_list[0].replace('[','')
    pre_list[0] = pre_list[0].replace(']','')
    print("-----------------------")
    print(j)
    nums_list = []
    
    for i in range(0,len(pre_list[0].split(","))):
        num = float(pre_list[0].split(",")[i])
        num = format(num,'.5f')
        nums_list.append(num)
        print(num)
    test = pd.read_csv('test2.csv',encoding='utf-8',sep="\t")
    print(nums_list)

-----------------------
0
['0.01615', '0.03394', '0.03726', '0.06168', '0.01166', '0.01297', '0.02902', '0.03015', '0.01766', '0.03956', '0.02698', '0.00688', '0.04803', '0.00868', '0.02093', '0.03903', '0.01775', '0.00840', '0.04710', '0.00000', '0.07752', '0.02580', '0.00000', '0.01166', '0.04560', '0.04723', '0.03394', '0.05102', '0.00609', '0.01166', '0.05102', '0.04545', '0.04545', '0.01284', '0.00000', '0.00000', '0.00000', '0.02089']
-----------------------
1
['0.04067', '0.00046', '0.08616', '0.07185', '0.05124', '0.12370', '0.00354', '0.12163', '0.09605', '0.00467', '0.04916', '0.07890', '0.03345', '0.05591', '0.05672', '0.00190', '0.01949', '0.03856', '0.00037', '0.06556']
-----------------------
2
['0.01725', '0.10476', '0.01004', '0.10850', '0.05310', '0.00000', '0.09613', '0.29350', '0.13502', '0.06533', '0.00013', '0.11625']
-----------------------
3
['0.36851', '0.36851', '0.00000', '0.00000', '0.26298']
-----------------------
4
['0.01102', '0.00938', '0.01039', '0.0375

['0.00001', '0.00179', '0.01047', '0.01130', '0.00476', '0.00754', '0.00887', '0.00993', '0.00119', '0.00582', '0.00419', '0.00966', '0.00601', '0.00501', '0.01666', '0.00437', '0.00486', '0.00859', '0.00001', '0.00474', '0.01846', '0.01009', '0.00000', '0.00012', '0.00142', '0.00022', '0.00410', '0.00186', '0.01566', '0.00000', '0.00689', '0.01243', '0.01692', '0.00684', '0.01197', '0.01611', '0.00078', '0.00850', '0.00014', '0.00760', '0.00765', '0.00757', '0.00360', '0.01232', '0.01071', '0.01428', '0.01366', '0.00399', '0.00109', '0.01241', '0.00493', '0.00187', '0.01633', '0.00953', '0.00034', '0.00060', '0.00486', '0.01019', '0.00531', '0.00514', '0.00620', '0.00145', '0.00540', '0.00908', '0.00591', '0.00036', '0.01587', '0.00470', '0.00514', '0.00410', '0.00116', '0.00240', '0.00195', '0.01140', '0.00164', '0.00720', '0.00094', '0.00161', '0.00109', '0.00543', '0.00703', '0.00146', '0.00906', '0.00514', '0.00109', '0.02250', '0.00003', '0.00633', '0.00365', '0.00739', '0.00736'

['0.01060', '0.01455', '0.02129', '0.01219', '0.00911', '0.00000', '0.01631', '0.02118', '0.01192', '0.00518', '0.00010', '0.01681', '0.00757', '0.01122', '0.01969', '0.01568', '0.01821', '0.01606', '0.01441', '0.00800', '0.03247', '0.00496', '0.02203', '0.00439', '0.01129', '0.00496', '0.01455', '0.01803', '0.00122', '0.00840', '0.00038', '0.01163', '0.00409', '0.01283', '0.03348', '0.02976', '0.02756', '0.01191', '0.00000', '0.01798', '0.01455', '0.00237', '0.01583', '0.00840', '0.02482', '0.02442', '0.01905', '0.00042', '0.00808', '0.02368', '0.00000', '0.00007', '0.00840', '0.01313', '0.00739', '0.02052', '0.00622', '0.00842', '0.01786', '0.01649', '0.01013', '0.01129', '0.01441', '0.01320', '0.00313', '0.01024', '0.00927', '0.00796', '0.01950', '0.02153', '0.03615', '0.03077', '0.01313', '0.01292', '0.02015', '0.00078', '0.00328', '0.00000']
-----------------------
591
['0.00040', '0.52591', '0.11219', '0.36150']
-----------------------
592
['0.19804', '0.08192', '0.11612', '0.073

['0.02882', '0.00474', '0.00289', '0.00224', '0.02146', '0.00290', '0.04109', '0.01216', '0.00379', '0.05149', '0.00382', '0.01648', '0.02252', '0.03336', '0.02092', '0.00224', '0.03345', '0.00000', '0.04302', '0.00437', '0.00559', '0.01895', '0.01169', '0.01953', '0.00337', '0.02088', '0.00842', '0.02743', '0.01791', '0.01559', '0.01119', '0.00216', '0.03065', '0.03049', '0.00955', '0.01524', '0.00247', '0.01819', '0.01998', '0.00784', '0.00212', '0.00319', '0.01219', '0.01035', '0.02445', '0.01188', '0.00023', '0.00474', '0.01200', '0.01240', '0.00000', '0.01642', '0.01531', '0.00561', '0.02292', '0.00624', '0.02508', '0.01239', '0.01189', '0.01359', '0.01535', '0.00744', '0.00983', '0.01865', '0.03323', '0.00004', '0.03215', '0.00325', '0.00817']
-----------------------
802
['0.14186', '0.08025', '0.04235', '0.09086', '0.09055', '0.01522', '0.02764', '0.03815', '0.00160', '0.10192', '0.04533', '0.06401', '0.07530', '0.06379', '0.00148', '0.06317', '0.05653']
-----------------------


-----------------------
1084
['0.23715', '0.24963', '0.18430', '0.01090', '0.02154', '0.00636', '0.29012']
-----------------------
1085
['0.14311', '0.02950', '0.10828', '0.02856', '0.03597', '0.00005', '0.00000', '0.04459', '0.04441', '0.12528', '0.07340', '0.00050', '0.10145', '0.04448', '0.06652', '0.03600', '0.02293', '0.00840', '0.01563', '0.04143', '0.02934', '0.00015']
-----------------------
1086
['0.00650', '0.00000', '0.00000', '0.00953', '0.00000', '0.02548', '0.00016', '0.00083', '0.00380', '0.00000', '0.00901', '0.02548', '0.00022', '0.00000', '0.00000', '0.00000', '0.02548', '0.00000', '0.02548', '0.00000', '0.00000', '0.00003', '0.02548', '0.00699', '0.00380', '0.01119', '0.00000', '0.00560', '0.00000', '0.00722', '0.00901', '0.01119', '0.02548', '0.02961', '0.00000', '0.00000', '0.00000', '0.00342', '0.00579', '0.02576', '0.00007', '0.00032', '0.00000', '0.02619', '0.02942', '0.00000', '0.00940', '0.00000', '0.00068', '0.00901', '0.00355', '0.00560', '0.00560', '0.00007

['0.00676', '0.00731', '0.00050', '0.00995', '0.00870', '0.00190', '0.00402', '0.00499', '0.01656', '0.00912', '0.00568', '0.00494', '0.00894', '0.00129', '0.01793', '0.01811', '0.00718', '0.00003', '0.00214', '0.00024', '0.00200', '0.00489', '0.00160', '0.00736', '0.00364', '0.00767', '0.00000', '0.00001', '0.00000', '0.00237', '0.00000', '0.00752', '0.00825', '0.00503', '0.00488', '0.00437', '0.00008', '0.02577', '0.00197', '0.00141', '0.01970', '0.01053', '0.02036', '0.01167', '0.01759', '0.00494', '0.00127', '0.01783', '0.00144', '0.00381', '0.00491', '0.01076', '0.01394', '0.00000', '0.00556', '0.00487', '0.00413', '0.01188', '0.00875', '0.00177', '0.00729', '0.02319', '0.00149', '0.00630', '0.00691', '0.00196', '0.00796', '0.01706', '0.00947', '0.00920', '0.00364', '0.00976', '0.00455', '0.00293', '0.00822', '0.00533', '0.01172', '0.00777', '0.01735', '0.00178', '0.02105', '0.00000', '0.01579', '0.00687', '0.00038', '0.00762', '0.00505', '0.00831', '0.00488', '0.01420', '0.00006'

['0.00000', '0.14430', '0.01165', '0.09873', '0.04289', '0.01436', '0.11310', '0.00065', '0.13111', '0.00000', '0.12778', '0.01568', '0.09873', '0.03242', '0.13250', '0.00623', '0.01481', '0.01469', '0.00000', '0.00036']
-----------------------
1596
['0.06288', '0.00000', '0.16506', '0.06288', '0.10218', '0.07658', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000', '0.06288', '0.10218', '0.06288', '0.00119', '0.20173', '0.09955']
-----------------------
1597
['0.00000', '0.00487', '0.00000', '0.00000', '0.02454', '0.06178', '0.00000', '0.09652', '0.00000', '0.00000', '0.00378', '0.00000', '0.03696', '0.00487', '0.00000', '0.06178', '0.00000', '0.00000', '0.06178', '0.06209', '0.00378', '0.00064', '0.00000', '0.06178', '0.06949', '0.00000', '0.00369', '0.00369', '0.00000', '0.00000', '0.13126', '0.00000', '0.06178', '0.00000', '0.00000', '0.02454', '0.00000', '0.06209', '0.00000', '0.06178', '0.00000', '0.09652', '0.00000', '0.00000', '0.00000', '0.00000', '0.00000']
---------------

-----------------------
1902
['0.01206', '0.00095', '0.00377', '0.00063', '0.00833', '0.00284', '0.03800', '0.01122', '0.01238', '0.00284', '0.01748', '0.00003', '0.02804', '0.04285', '0.00256', '0.00260', '0.03803', '0.02386', '0.01035', '0.00917', '0.00009', '0.00000', '0.00000', '0.00000', '0.02648', '0.00920', '0.01035', '0.00993', '0.01943', '0.03306', '0.02356', '0.01508', '0.01335', '0.01008', '0.00567', '0.03521', '0.00801', '0.01256', '0.00710', '0.02206', '0.02663', '0.01683', '0.00285', '0.00726', '0.00000', '0.01088', '0.00355', '0.03092', '0.03924', '0.00101', '0.06003', '0.00537', '0.00317', '0.00000', '0.00032', '0.00402', '0.01078', '0.02186', '0.02055', '0.00284', '0.01225', '0.02428', '0.01940', '0.00609', '0.01683', '0.00475', '0.00301', '0.02493', '0.00479', '0.00004', '0.00260', '0.02200', '0.00704', '0.00803', '0.01940', '0.02348', '0.00377']
-----------------------
1903
['0.01043', '0.25803', '0.00000', '0.17903', '0.04711', '0.08942', '0.29169', '0.12429']
-----